# Проинсталирую библиотеку

In [ ]:
# installing libraries
!pip install openai -q # для ChatGPT

#Этапы реализации

*Этап 1: Сбор базы знаний*
Собираю знания по медицине с Википедии и сохраняю их в CSV-файл.

In [ ]:
# Импорт библиотек
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd

In [ ]:
# Список статей для сбора (можете расширить)
pages = [
    'https://ru.wikipedia.org/wiki/Медицина',
    # Добавьте дополнительные страницы
]

def parse_wiki_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    title = soup.find('h1', id='firstHeading').text.strip()
    content = '\n'.join([p.text for p in soup.find_all('p')])
    return {'title': title, 'content': content}

with open('medicine_knowledge.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Content'])

    for page_url in pages:
        data = parse_wiki_page(page_url)
        writer.writerow([data['title'], data['content']])

In [ ]:
# Читаем CSV-файл в DataFrame
df = pd.read_csv('medicine_knowledge.csv', encoding='utf-8')

# Выводим заголовки
print("Заголовки файла:", df.columns.tolist())

# Выводим первые пять строк
print(df.head())

Заголовки файла: ['Title', 'Content']
      Title                                            Content
0  Медицина  Медици́на (от лат. medicīna[1], от словосочета...


Этап 2: Создание Telegram-бота с библиотекой Aiogram

In [ ]:
!pip install aiogram -q # для тг-бота

In [ ]:
# Установим библиотеку nest_asyncio для работы в Colabe
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import asyncio
from aiogram import Bot, Dispatcher, types
from aiogram.types import Message
from aiogram.filters import Command
from aiogram import Router

# Замените этот токен на ваш собственный токен бота
API_TOKEN = 'Ваш токен с ТГ - замените'

# Создаем экземпляр бота
bot = Bot(token=API_TOKEN)

# Создаем экземпляр диспетчера
dp = Dispatcher()

# Создаем экземпляр роутера
router = Router()

# Обработчик команды /start
@router.message(Command("start"))
async def send_welcome(message: Message):
    """
    Отправляет приветственное сообщение пользователю, когда он использует команду /start.
    """
    await message.answer("Привет! Я медицинский бот, всё что касается медицины и Твоего здоровья, я смогу тебе помочь найти информацию! И даже смогу дать рекомендации.")

async def main():
    """
    Основная функция для запуска бота.
    """
    # Регистрация обработчиков
    dp.include_router(router)

    # Запускаем диспетчер
    await dp.start_polling(bot)

if __name__ == '__main__':
    # Запускаем основную функцию
    asyncio.run(main())


In [ ]:
Создание простого бота:



Этап 3: Интеграция с OpenAI API

Подключаем OpenAI API для обработки запросов пользователей с использованием заранее подготовленной базы знаний.

Установите модуль openai:

pip install openai

Код интеграции:

import os
import pandas as pd
import openai

OPENAI_API_KEY = 'YOUR_OPENAI_API_KEY'  # Ваш ключ OpenAI
openai.api_key = OPENAI_API_KEY

knowledge_base = pd.read_csv('medicine_knowledge.csv')

def generate_response(prompt):
    messages = []
    for _, row in knowledge_base.iterrows():
        if prompt.lower() in row['Title'].lower() or prompt.lower() in row['Content'].lower():
            messages.append({"role": "system", "content": f"{row['Title']} {row['Content']}"})

    messages.append({"role": "user", "content": prompt})
    completion = openai.ChatCompletion.create(
      model="Anthropic: Claude 3 Haiku",
      messages=messages,
      max_tokens=150
    )
    return completion.choices[0].message.content

@dp.message_handler()
async def handle_message(message: types.Message):
    user_input = message.text
    bot_response = generate_response(user_input)
    await message.answer(bot_response)

Этап 4: Метод дообучения с помощью Search-Ask

Для дообучения модели используем метод Search-Ask, который позволяет получать дополнительную релевантную информацию из баз знаний перед генерацией ответа.

Шаги:

Поиск наиболее релевантных документов из базы знаний по запросу пользователя.
Подготовка контекста на основе найденных документов.
Передача контекста и вопроса модели OpenAI для генерации точного ответа.

Пример доработанного блока обработки запросов:

def find_relevant_docs(query):
    relevant_docs = []
    for index, row in knowledge_base.iterrows():
        if query.lower() in row['Title'].lower() or query.lower() in row['Content'].lower():
            relevant_docs.append(row['Content'])
    return relevant_docs[:3]  # Берём первые три документа

def create_context(relevant_docs):
    context = "\n\n".join(relevant_docs)
    return context

def ask_gpt_with_context(context, question):
    full_prompt = f"""
    Context:\n{context}\nQuestion:{question}
    """
    return generate_response(full_prompt)

@dp.message_handler()
async def handle_message(message: types.Message):
    user_input = message.text
    relevant_docs = find_relevant_docs(user_input)
    context = create_context(relevant_docs)
    final_answer = ask_gpt_with_context(context, user_input)
    await message.answer(final_answer)


Таким образом, мы получили рабочий прототип медицинского Telegram-бота с интеграцией OpenAI и механизмом улучшения качества ответов путем дообучения через предварительный поиск соответствующих медицинских сведений из предварительно собранной базы знаний.

Теперь вы можете продолжить развивать проект, улучшив обработку вопросов, увеличив количество источников знаний, расширяя возможности модели и оптимизируя производительность.